In [1]:
import os 
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
tfe.enable_eager_execution()

In [4]:
# TensorFlow MNIST
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(1000) #Accuracy:0.8979
# tf.set_random_seed(57)
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
dim_hidden = 1024

In [6]:
layer_cnn0 = tf.layers.Conv2D(32, 5, activation = tf.nn.relu)

In [7]:
layer_pool2x2 = tf.layers.MaxPooling2D(2, 2)

In [8]:
layer_cnn1 = tf.layers.Conv2D(64, 5, padding="same", activation = tf.nn.relu)

In [9]:
layer_pool3x3 = tf.layers.MaxPooling2D(3, 3)

In [10]:
layer_cnn2 = tf.layers.Conv2D(80, 5,  padding="same", activation = tf.nn.relu)

In [11]:
layer_pool4x4 = tf.layers.MaxPooling2D(4, 4)

In [12]:
layer_cnn3 = tf.layers.Conv2D(64, 5,  padding="same", activation = tf.nn.relu)

In [13]:
layer_flatten = tf.layers.Flatten()

In [14]:
layer_fc0 = tf.layers.Dense(dim_hidden, activation = tf.nn.relu)

In [15]:
layer_fc1 = tf.layers.Dense(3*dim_hidden/4, activation = tf.nn.relu)

In [16]:
layer_fc2 = tf.layers.Dense(dim_hidden/2, activation = tf.nn.relu)

In [17]:
layer_fc3 = tf.layers.Dense(dim_hidden/4, activation = tf.nn.relu)

In [18]:
layer_fc4 = tf.layers.Dense(dim_hidden/4, activation = tf.nn.relu)

In [19]:
layer_dropout = tf.layers.Dropout(rate=0.60) # dropout rate is 0.75. Retain 0.25

In [20]:
layer_fc5 = tf.layers.Dense(10, activation = tf.nn.relu) # 1

In [21]:
# forward propagation
def prediction(X, training):
    values = tf.constant(X)
    values = layer_pool4x4(values) # this must be the first layer
    values = layer_cnn0(values)
    values = layer_cnn1(values)
#     values = layer_cnn2(values)
#     values = layer_pool2x2(values)

#     values = layer_cnn3(values)
    
    values = layer_flatten(values)
    values = layer_fc0(values)
    values = layer_fc1(values)

    values = layer_fc2(values)

#     values = layer_fc3(values)
#     values = layer_fc4(values)
    values = layer_dropout(values, training=training)
    values = layer_fc5(values)
    return values

In [22]:
# cross entropy loss
def loss(X, y, training):
    logits = prediction(X, training)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = logits)
    loss = tf.reduce_mean(loss)
    return loss


In [23]:
def binary_accuracy(X, y):
    logits = prediction(X, training = False)
    predict = tf.argmax(logits, 1).numpy()
    target = np.argmax(y, 1)
    binary_accuracy = np.sum(predict == target)/len(target)
    return(binary_accuracy)

In [24]:
X_validation = mnist.validation.images
y_validation = mnist.validation.labels
X_validation = X_validation.reshape([-1,28,28,1])

In [25]:
def v_binary_accuracy() :
    return(binary_accuracy(X_validation, y_validation))

In [26]:
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3, epsilon=1e-9)

In [27]:
batch_size = 50
iters = 2000

In [28]:
for i in range(iters):
    X, y = mnist.train.next_batch(batch_size)
    X = X.reshape([-1,28,28,1])
    optimizer.minimize(lambda: loss(X, y, True))

    if i % 100 == 0:
        batch_accuracy = binary_accuracy(X, y)
        validation_accuracy = v_binary_accuracy()
        print("batch %d, batch accuracy %.3f;  validation accuracy %.3f" %
                                (i, batch_accuracy, validation_accuracy))

batch 0, batch accuracy 0.180;  validation accuracy 0.089
batch 100, batch accuracy 0.720;  validation accuracy 0.736
batch 200, batch accuracy 0.820;  validation accuracy 0.788
batch 300, batch accuracy 0.860;  validation accuracy 0.855
batch 400, batch accuracy 0.900;  validation accuracy 0.872
batch 500, batch accuracy 0.820;  validation accuracy 0.875
batch 600, batch accuracy 0.900;  validation accuracy 0.882
batch 700, batch accuracy 0.880;  validation accuracy 0.897
batch 800, batch accuracy 0.880;  validation accuracy 0.886
batch 900, batch accuracy 0.920;  validation accuracy 0.881
batch 1000, batch accuracy 0.920;  validation accuracy 0.896
batch 1100, batch accuracy 0.900;  validation accuracy 0.890
batch 1200, batch accuracy 0.800;  validation accuracy 0.893
batch 1300, batch accuracy 0.960;  validation accuracy 0.897
batch 1400, batch accuracy 0.900;  validation accuracy 0.892
batch 1500, batch accuracy 0.920;  validation accuracy 0.907
batch 1600, batch accuracy 0.880;  v

In [29]:
# evaluate the result
X, y = mnist.test.images, mnist.test.labels
X = X.reshape([-1,28,28,1])
test_accuracy = binary_accuracy(X, y)
print("test accuracy %g" % (test_accuracy))

test accuracy 0.8979
